In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F
from pyspark.sql.functions import to_date
from pyspark.sql.functions import datediff


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('hw5') \
    .getOrCreate()


In [3]:
## Q1

spark.version

'3.3.2'

In [4]:
my_schema = types.StructType([
 types.StructField('dispatching_base_num', types.StringType(), True),
 types.StructField('pickup_datetime', types.TimestampType(), True),
 types.StructField('dropOff_datetime', types.TimestampType(), True),
 types.StructField('PUlocationID', types.IntegerType(), True),
 types.StructField('DOlocationID', types.IntegerType(), True),
 types.StructField('SR_Flag', types.StringType(), True),
 types.StructField('Affiliated_base_number', types.StringType(), True)
 ])

In [5]:
## Q2 Read in data to a dataframe

df = spark.read \
    .option("header", "true") \
    .schema(my_schema) \
    .csv(r'C:\Users\matth\Zoom Camp 2024\Module (5) Batch Processing\HW-5\Data\fhv_tripdata_2019-10.csv')

In [6]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
+--------------------+------------------

In [7]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [8]:
## Writing dataframe to Parquet

df.repartition(6).write.mode("overwrite").parquet(r'C:\Users\matth\Zoom Camp 2024\Module (5) Batch Processing\HW-5\Output_Data')

In [9]:
## Q4

df_lengths = df.withColumn("time_difference",datediff(df.pickup_datetime, df.dropOff_datetime))

df_with_time_difference = df.withColumn("time_difference_hours",(F.col("dropOff_datetime") - F.col("pickup_datetime")).cast("long")/3600)

# Find the maximum value of the new column
max_time_difference_hours = df_with_time_difference.agg(F.max("time_difference_hours")).collect()[0][0]

print("Maximum time difference (in hours):", max_time_difference_hours)


Maximum time difference (in hours): 631152.5


In [10]:
## Q6

df_zones = spark.read \
    .option("header", "true") \
    .option("inferSchema", True) \
    .csv(r'C:\Users\matth\Zoom Camp 2024\Module (5) Batch Processing\HW-5\Data\taxi_zone_lookup.csv')


#df_zones.printSchema()

df_zones.show(10)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 10 rows



In [11]:
df_zones.createOrReplaceTempView("zones_data")
df.createOrReplaceTempView("fhv_data") 

In [12]:
sql_query = """SELECT z.Zone, COUNT(f.PULocationID) AS PickupLocCount 
FROM FHV_data f 
JOIN zones_data z 
ON f.PUlocationID = z.LocationID 
GROUP BY z.Borough, z.Zone 
ORDER BY PickupLocCount 
LIMIT 1;"""

result = spark.sql(sql_query)
result.show()

+-----------+--------------+
|       Zone|PickupLocCount|
+-----------+--------------+
|Jamaica Bay|             1|
+-----------+--------------+

